## Import Dependencies

In [89]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time as time

## Read Comments
## Drop Unnessessary Column
## Use Ceiling function to Return the Rating as an Integer

In [2]:
comments = pd.read_csv('../data/large_sample.csv')
comments.drop(columns=['Unnamed: 0'], inplace=True)
comments.rating = np.ceil(comments.rating)

In [4]:
comments.head()

,commenter,gameID,rating,comment
0,172640,24068,7.0,Good: Unique take on the hidden role games. T...
1,86674,24068,7.0,A neat social deduction game with multiple tea...
2,10643,24068,7.0,Good hidden roles werewolf style game that can...
3,31171,24068,7.0,"Overall I hate Mafia/Werewolf, but this versio..."
4,165608,24068,7.0,Fun social deduction exercise that gets merrie...


## Remove Punctuation
## Remove Uppercase

In [7]:
## return a string in lower case
def make_it_lower(comment):
    new_comment = []
    for x in comment.split(' '):
        new_comment.append(x.lower())
    return ' '.join(new_comment)

## remove punctuation
def remove_punctuation(comment):
    table = str.maketrans(dict.fromkeys('@#$%^&*()_+=~`[{}]|\:;"<,>."/'))
    return comment.translate(table)

## replace punctuation
def replace_exclimation(comment):
    return comment.replace('!', ' <!> ').replace('?', ' <?> ')

In [8]:
comments['lower_comment'] = comments.comment.apply(lambda x: make_it_lower(x))
comments.drop(columns=['comment'], inplace=True)
comments.rename(columns={'lower_comment':'comment'}, inplace=True)
comments['remove_punctuation'] = comments.comment.apply(lambda x: remove_punctuation(x))
comments.drop(columns=['comment'], inplace=True)
comments.rename(columns={'remove_punctuation':'comment'}, inplace=True)
comments['replace_punctuation'] = comments.comment.apply(lambda x: replace_exclimation(x))
comments.drop(columns=['comment'], inplace=True)
comments.rename(columns={'replace_punctuation':'comment'}, inplace=True)

In [10]:
comments.head()

,commenter,gameID,rating,comment
0,172640,24068,7.0,good unique take on the hidden role games the...
1,86674,24068,7.0,a neat social deduction game with multiple tea...
2,10643,24068,7.0,good hidden roles werewolf style game that can...
3,31171,24068,7.0,overall i hate mafiawerewolf but this version ...
4,165608,24068,7.0,fun social deduction exercise that gets merrie...


## Drop Commenter and gameID

In [11]:
comments.drop(columns=['commenter', 'gameID'], inplace=True)

## Create a Dictionary of every word with its frequency

In [21]:
def create_count(comment):
    for x in comment.split(' '):
        if x in word_dictionary:
            word_dictionary[x] += 1
        else:
            word_dictionary[x] = 1
    return word_dictionary

In [33]:
%%time
word_dictionary = dict()
comments.comment.apply(create_count)

CPU times: user 13.4 s, sys: 525 ms, total: 14 s
Wall time: 14 s


In [36]:
word_dictionary

{'good': 142106,
 '': 1268570,
 'unique': 10304,
 'take': 31312,
 'on': 173501,
 'the': 1560992,
 'hidden': 6067,
 'role': 7607,
 'games': 143376,
 'and': 833047,
 'evil': 1586,
 'team': 4861,
 'win': 27673,
 'if': 112738,
 'they': 48130,
 'eliminate': 494,
 'each': 41510,
 'other': 65740,
 'where': 31856,
 'neutral': 631,
 'has': 81277,
 'objectives': 2148,
 'depending': 4230,
 'what': 68003,
 'was': 98404,
 'dealt': 1161,
 'bad': 24962,
 'component': 2621,
 'quality': 9785,
 'is': 549280,
 'lower': 2896,
 'end': 28146,
 'artwork': 13068,
 'lackluster': 329,
 'card': 61041,
 'stock': 2231,
 'flimsy': 419,
 'board': 45709,
 'design': 15571,
 'uninspired': 274,
 'playing': 79252,
 'this': 463479,
 'entertaining': 4234,
 'but': 374843,
 'its': 45123,
 'tough': 7061,
 'getting': 18428,
 'folks': 1971,
 'interested': 4969,
 'when': 70744,
 'newer': 1950,
 'have': 176728,
 'a': 999023,
 'much': 108750,
 'better': 64119,
 'graphical': 145,
 'presentation': 1090,
 'neat': 5886,
 'social': 460

In [37]:
len(word_dictionary)

246987

In [43]:
more_than_ten = {k: v for k, v in word_dictionary.items() if v > 10}
ten_or_less = {k: v for k, v in word_dictionary.items() if v <=10}

## Drop words that appear 10 or less times

In [40]:
print(len(more_than_ten))
print(len(ten_or_less))

27372
219615


In [41]:
%%time
sortedList=sorted(more_than_ten.values())
sorted_dict = dict()
for sortedKey in sortedList:
    for key, value in more_than_ten.items():
        if value==sortedKey:
            sorted_dict[key]=value

CPU times: user 2min, sys: 227 ms, total: 2min 1s
Wall time: 2min 1s


In [42]:
array_of_words = []
for key, value in sorted_dict.items():
    temp = [key,value]
    array_of_words.append(temp)

In [50]:
len(array_of_words)

27372

## Make reports of the frequency each word has in comments of different ratings

In [48]:
def word_count(comment, word, number):
    if word in comment.split(' '):
        if word in number:
            number[word] += comment.split(' ').count(word)
        else:
            number[word] = comment.split(' ').count(word)

In [92]:
def make_report(w, z, dictionary, dataframe):
    dict_list = dictionary
    testing_df = dataframe.copy()
    labels = ['word', 'average/std', 'percent_average', 'percent_std', 'rating', 'word_count', 'divisor', 'percent', 'time', 'type']
    to_return = pd.DataFrame(columns=labels)
    for word in dict_list[int(w):int(z)]:
        start_time = time.time()
        to_append = []
        std = 0
        average = 0
        for y in range(1,11):
            number = dict()
            testing_df.loc[testing_df.rating == y].comment.apply(word_count, word=word[0], number=number)
            divisor = testing_df.loc[testing_df.rating == y].comment.count()
            if word[0] in number:
                percent = number[word[0]]/divisor
            else:
                percent = 0
                number[word[0]] = 0
            to_append.append([word[0], None, None, None, y, number[word[0]], divisor, percent, None, 0])
        for_array = []
        for value in to_append:
            for_array.append(value[7])
        x = np.array(for_array)
        array1 = [[word[0], (np.sum(x)/10)/(np.std(x)), np.sum(x)/10, np.std(x), None, None, None, None, time.time()-start_time, 1]]
        for x in to_append:
            array1.append(x)
        inbetween_df = pd.DataFrame.from_records(array1, columns=labels)
        to_return = pd.concat([to_return, inbetween_df])
    return to_return

In [51]:
def write_csv(low, high, df, count):
    df.to_csv('new_report_for_{}_{}_to_{}.csv'.format(str(count).zfill(2), low, high))

In [52]:
def loop_write_report(dictionary, dataframe, number, count):
    num = int(number)
    size = int(count)
    first = 0
    counter = 0
    while first < num - 1:
        counter += 1
        if first + size < num - 2:
            report = make_report(first, first + size, dictionary, dataframe)
            write_csv(first, first + size, report, counter)
            print('dataframe written from {} to {}'.format(first, first + size))
            first = first + size + 1
        else:
            report = make_report(first, num - 1, dictionary, dataframe)
            write_csv(first, num - 1, report, counter)
            print('dataframe written from {} to {}'.format(first, num - 1))
            print('last dataframe')
            break

In [87]:
def loop_combine_reports_delete_old_reports(dictionary, dataframe, number, count):
    loop_write_report(dictionary, dataframe, number, count)
    glob_reports()    

In [74]:
def glob_reports():
    counter = 0
    df = pd.DataFrame()
    for report in glob.glob('new_report_for_*'):
        to_add = pd.read_csv(report)
        df = pd.concat([df, to_add])
        os.remove(report)
    df.to_csv('to_delete.csv')
        

In [94]:
len(array_of_words)

27372

In [95]:
loop_combine_reports_delete_old_reports(dictionary=array_of_words, dataframe=comments, number=len(array_of_words), count=100)

dataframe written from 0 to 100
dataframe written from 101 to 201
dataframe written from 202 to 302
dataframe written from 303 to 403
dataframe written from 404 to 504
dataframe written from 505 to 605
dataframe written from 606 to 706
dataframe written from 707 to 807
dataframe written from 808 to 908
dataframe written from 909 to 1009
dataframe written from 1010 to 1110
dataframe written from 1111 to 1211
dataframe written from 1212 to 1312
dataframe written from 1313 to 1413
dataframe written from 1414 to 1514
dataframe written from 1515 to 1615
dataframe written from 1616 to 1716
dataframe written from 1717 to 1817
dataframe written from 1818 to 1918
dataframe written from 1919 to 2019
dataframe written from 2020 to 2120
dataframe written from 2121 to 2221
dataframe written from 2222 to 2322
dataframe written from 2323 to 2423
dataframe written from 2424 to 2524
dataframe written from 2525 to 2625
dataframe written from 2626 to 2726
dataframe written from 2727 to 2827
dataframe wri

dataframe written from 22422 to 22522
dataframe written from 22523 to 22623
dataframe written from 22624 to 22724
dataframe written from 22725 to 22825
dataframe written from 22826 to 22926
dataframe written from 22927 to 23027
dataframe written from 23028 to 23128
dataframe written from 23129 to 23229
dataframe written from 23230 to 23330
dataframe written from 23331 to 23431
dataframe written from 23432 to 23532
dataframe written from 23533 to 23633
dataframe written from 23634 to 23734
dataframe written from 23735 to 23835
dataframe written from 23836 to 23936
dataframe written from 23937 to 24037
dataframe written from 24038 to 24138
dataframe written from 24139 to 24239
dataframe written from 24240 to 24340
dataframe written from 24341 to 24441
dataframe written from 24442 to 24542
dataframe written from 24543 to 24643
dataframe written from 24644 to 24744
dataframe written from 24745 to 24845
dataframe written from 24846 to 24946
dataframe written from 24947 to 25047
dataframe wr

In [96]:
wow = pd.read_csv('to_delete.csv')

In [102]:
wow.loc[wow.type == 1].loc[wow.loc[wow.type == 1]['average/std']<1.5].sort_values(by=['time'],ascending=False)

,Unnamed: 0,Unnamed: 0.1,word,average/std,percent_average,percent_std,rating,word_count,divisor,percent,time,type
95447,847,0,lauded,0.669338,0.000050,0.000075,NaN,NaN,NaN,NaN,8925.841075,1
52734,1034,0,gloomhaven,1.128480,0.000059,0.000053,NaN,NaN,NaN,NaN,7209.253198,1
88484,484,0,bgcolor000000colorffffffsoloxcolorbgcolor,0.690419,0.000006,0.000009,NaN,NaN,NaN,NaN,7207.159381,1
226897,297,0,composite,0.998576,0.000045,0.000045,NaN,NaN,NaN,NaN,7205.270397,1
13134,1034,0,mists,0.891603,0.000022,0.000024,NaN,NaN,NaN,NaN,7203.623691,1
215787,187,0,deckpool,0.785894,0.000013,0.000017,NaN,NaN,NaN,NaN,5790.936311,1
182314,814,0,1944,0.488486,0.000036,0.000073,NaN,NaN,NaN,NaN,5746.254380,1
95557,957,0,wrenching,1.140427,0.000041,0.000036,NaN,NaN,NaN,NaN,5257.979689,1
95370,770,0,boast,0.957479,0.000023,0.000024,NaN,NaN,NaN,NaN,4175.041074,1
216084,484,0,lighthouse,0.541551,0.000041,0.000075,NaN,NaN,NaN,NaN,3774.761425,1


In [103]:
wow.loc[wow.word == '4-player-']

,Unnamed: 0,Unnamed: 0.1,word,average/std,percent_average,percent_std,rating,word_count,divisor,percent,time,type
211277,77,0,4-player-,0.333333,0.000005,0.000015,NaN,NaN,NaN,NaN,2211.857467,1
211278,78,1,4-player-,NaN,NaN,NaN,1.0,0.0,3970.0,0.00000,NaN,0
211279,79,2,4-player-,NaN,NaN,NaN,2.0,0.0,8613.0,0.00000,NaN,0
211280,80,3,4-player-,NaN,NaN,NaN,3.0,0.0,16524.0,0.00000,NaN,0
211281,81,4,4-player-,NaN,NaN,NaN,4.0,0.0,29749.0,0.00000,NaN,0
211282,82,5,4-player-,NaN,NaN,NaN,5.0,0.0,52767.0,0.00000,NaN,0
211283,83,6,4-player-,NaN,NaN,NaN,6.0,0.0,110989.0,0.00000,NaN,0
211284,84,7,4-player-,NaN,NaN,NaN,7.0,0.0,197396.0,0.00000,NaN,0
211285,85,8,4-player-,NaN,NaN,NaN,8.0,11.0,222182.0,0.00005,NaN,0
211286,86,9,4-player-,NaN,NaN,NaN,9.0,0.0,131276.0,0.00000,NaN,0


In [1]:
def write_normialized_statistics(series, df):
    mean_total = 0
    median_total = 0
    mode_total = 0
    mode = 0
    counter = 0
    stop_median = 0
    for x in series:
        if counter > 0:
            mean_total += counter*x
            median_total += x
            if (median_total >= 50 and stop_median == 0):
                df.loc[(df.word == series.name) & (df.type == 1), 'median'] = counter
                stop_median = 1
            if x > mode_total:
                mode_total = x
                mode = counter
        counter += 1
    df.loc[(df.word == series.name) & (df.type == 1), 'mean'] = mean_total/100
    df.loc[(df.word == series.name) & (df.type == 1), 'mode'] = mode